In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Reading and preprocessing data

First we need to convert dicom files to objects that we can use; CSV files, jpg or png images e.g. To do this, we can use pydicom library for reading dicom files and then converting them. For more information you can visit these links:

[https://asvcode.github.io/MedicalImaging/medical_imaging/dicom/fastai/2020/04/28/Medical-Imaging-Using-Fastai.html](http://)

[https://pydicom.github.io/pydicom/stable/tutorials/dataset_basics.html](http://)

In [ ]:
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
from fastai.medical.imaging import *

import pydicom, kornia, cv2

train_dcm_path = get_dicom_files("../input/unifesp-x-ray-body-part-classifier/train")
test_dcm_path = get_dicom_files("../input/unifesp-x-ray-body-part-classifier/test")

print("train len:", len(train_dcm_path), "\ntest len:", len(test_dcm_path))

Now we can convert some parts of dicom file to pandas dataset. Thanks to Mr.Bombonato, he convert these dicom files to csv files you can find here:

[https://www.kaggle.com/datasets/ibombonato/unifesp-xray-body-part-dicom-metadata-csv](http://)

In [ ]:
train_dcm2csv_df = pd.read_csv('../input/unifesp-xray-body-part-dicom-metadata-csv/dicom_metadata_train.csv')
train_dcm2csv_df

Furthermore, there's another csv file in the main dataset that includes "SOP IDs" and "targets".

In [ ]:
train_df = pd.read_csv('../input/unifesp-x-ray-body-part-classifier/train.csv')
train_df

In [ ]:
train_df.info()

In [ ]:
train_df['Target'].unique()

The labels are represented as integers that map to the following:


* Abdomen = 0
* Ankle = 1
* Cervical Spine = 2
* Chest = 3
* Clavicles = 4
* Elbow = 5
* Feet = 6
* Finger = 7
* Forearm = 8
* Hand = 9
* Hip = 10
* Knee = 11
* Lower Leg = 12
* Lumbar Spine = 13
* Others = 14
* Pelvis = 15
* Shoulder = 16
* Sinus = 17
* Skull = 18
* Thigh = 19
* Thoracic Spine = 20
* Wrist = 21

As you see, some of dicom files have more that one class (target)! So, we have a multilabel classification problem. This is my strategy:

First we add all labels to the "train_df" as new columns by their orders and set all to zero. Then by considering the numbers in the target column of each record, we replace 1 in the columns related that number. 

For example, suppose that target have two numbers: "1 , 6". These numbers represent "ankle, feet". We replace 1 in "ankle" and "feet" columns of that record.

In [ ]:
train_df['Abdomen'] = 0
train_df['Ankle'] = 0
train_df['Cervical Spine'] = 0
train_df['Chest'] = 0
train_df['Clavicles'] = 0
train_df['Elbow'] = 0
train_df['Feet'] = 0
train_df['Finger'] = 0
train_df['Forearm'] = 0
train_df['Hand'] = 0
train_df['Hip'] = 0
train_df['Knee'] = 0
train_df['Lower Leg'] = 0
train_df['Lumbar Spine'] = 0
train_df['Others'] = 0
train_df['Pelvis'] = 0
train_df['Shoulder'] = 0
train_df['Sinus'] = 0
train_df['Skull'] = 0
train_df['Thigh'] = 0
train_df['Thoracic Spine'] = 0
train_df['Wrist'] = 0
train_df

In [ ]:
labels = ['Abdomen', 'Ankle', 'Cervical Spine', 'Chest', 'Clavicles', 'Elbow', 'Feet', 'Finger', 'Forearm', 'Hand', 'Hip', 'Knee', 'Lower Leg', 'Lumbar Spine', 'Others', 'Pelvis', 'Shoulder', 'Sinus', 'Skull', 'Thigh', 'Thoracic Spine', 'Wrist']

for i in range(len(train_df)):
    lbl_list = train_df.Target[i].split()
    
    for j in lbl_list:
        train_df.loc[i, labels[int(j)]] = 1
        

train_df[:10]

Now, we merge two dataframs based on "SOPInstanceUID" columns. So we have an integrated dataframe which help us to create "X" and "y" of model much easier.

In [ ]:
train_merged_df = pd.merge(train_dcm2csv_df, train_df, on='SOPInstanceUID')
train_merged_df

An example of converting dicom file to jpg image:

In [ ]:
tmp = train_dcm_path[np.random.randint(0, 1737)].dcmread()
dcm_img = tmp.pixel_array.astype(float)
img = np.uint8((np.maximum(dcm_img,0)/dcm_img.max())*255)
plt.imshow(img, cmap='gray')

This step is the most time consuming part of the code; 

First we should reading dicom files and convert them to jpg images but there is a big problem; The size of dicom images are too big; for example 3000 * 4000 pixels! There isn't enough memory so we should resize all of them to 128 * 128 pixels but unfortunately it will have consequences like information loss. If you have enough memory, you can resize them to 512 * 512 pixels for example. Anyway, the images create X array for training.

The "y" of model, is an array of 1738 * 22. In another words, the label of each image (each SOPInstanceUID) is a vector contains 22 binary numbers that represent 22 labels (body parts). 

In [ ]:
import cv2
from matplotlib import pyplot as plt

X = []
y = []
for i in range(len(train_dcm_path)):
    tmp = train_dcm_path[i].dcmread()
    
    dcm_img = tmp.pixel_array.astype(float)
    rescaled_img = np.uint8((np.maximum(dcm_img,0)/dcm_img.max())*255)
    image = cv2.resize(rescaled_img, dsize=(128,128), interpolation=cv2.INTER_AREA)/255 #resizing and normalizing
    X.append(np.expand_dims(image,axis=-1))
    
    uid = str(tmp['SOPInstanceUID'].value)
    y.append(np.ndarray.flatten(np.array(train_merged_df.loc[train_merged_df['SOPInstanceUID'] == uid][train_merged_df.columns[64:]])))
    
X = np.array(X)
y = np.array(y)

# Training and testing custom CNN model

First we should split X and y data to train and test data.

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

To avoid of overfitting, we can use "EarlyStopping" callback; So we need to split train data to "train" and "validation" parts.

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [ ]:
### A sample of X_train and related y_train ###

plt.imshow(X_train[7], cmap='gray')
print(y_train[7])

Now, we can build our CNN model.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import InputLayer, Conv2D, Dropout, MaxPooling2D, Flatten, Dense

cnn_model = Sequential()
cnn_model.add(InputLayer(input_shape=(128,128,1)))
              
cnn_model.add(Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same'))
#cnn_model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
cnn_model.add(MaxPooling2D((2, 2)))

cnn_model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
#cnn_model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
cnn_model.add(MaxPooling2D((2, 2)))

cnn_model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
cnn_model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
cnn_model.add(MaxPooling2D((2, 2)))

cnn_model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same'))
cnn_model.add(MaxPooling2D((2, 2)))

cnn_model.add(Flatten())
cnn_model.add(Dense(4096, activation="relu"))
cnn_model.add(Dense(2048, activation="relu"))
cnn_model.add(Dense(22, activation="sigmoid"))

cnn_model.compile(optimizer='adam', loss='BinaryCrossentropy', metrics=['accuracy'])
cnn_model.summary()

In [ ]:
history = cnn_model.fit(x = X_train,
                        y = y_train,
                        batch_size = 8,
                        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)],
                        validation_data = (X_val, y_val),
                        verbose = 1,
                        epochs = 50)

Plotting results of trained model involvs accuracy, validation accuracy, loss and validation loss

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(13,5))

ax[0].plot(history.history['accuracy'])
ax[0].plot(history.history['val_accuracy'])
ax[0].set_title('model accuracy')
ax[0].set_ylabel('accuracy')
ax[0].set_xlabel('epoch')
ax[0].legend(['train', 'val'], loc='upper left')

ax[1].plot(history.history['loss'][1:])
ax[1].plot(history.history['val_loss'])
ax[1].set_title('model loss')
ax[1].set_ylabel('val_loss')
ax[1].set_xlabel('epoch')
ax[1].legend(['train', 'val'], loc='upper right')

fig.show()

The accuracy of the model on test data:

In [ ]:
cnn_model.evaluate(x = X_test,
                   y = y_test,
                   verbose=1)

# Prediction

First select a dicom file from test data and convert it to jpg image same as training phase. Then pass it to "predict" function and get output.

After some tests, I set threshold = 0.1 which means if score is greater than 0.1, the related label is in that test image. 

In [ ]:
test_candid = test_dcm_path[np.random.randint(0, 742)].dcmread()
dcm_img = test_candid.pixel_array.astype(float)
converted_img = np.uint8((np.maximum(dcm_img,0)/dcm_img.max())*255)
resized_img = cv2.resize(converted_img, dsize=(128,128), interpolation=cv2.INTER_AREA)/255
test_img = np.expand_dims(resized_img, axis=-1)
plt.imshow(test_img, cmap='gray')

image = np.expand_dims(test_img, axis=0)
prdct_arr = cnn_model.predict(image)

body_parts = []
prdct_arr = prdct_arr.flatten()
for i in range(len(prdct_arr)):
    if (prdct_arr[i] > 0.1):
        body_parts.append(labels[i])

print(body_parts)